In [ ]:
import ccxt
import pandas as pd
binance = ccxt.binance({
    'apiKey':'heJCdTPU31a1u0zmBcv1f20V1EZnax23O6njULZn2cutz41YtydEFhcOJcOg8dAm',
    'secret':'ESuH2Y4ILi061nQJ6MXGfGD7Qct0OEEl9wf0mI8KMH8SQiCzQx4EubfhJvrHEzZB'
})
binance.load_markets()
new_pairs = [p for p in new_pairs if p in binance.symbols]
len(new_pairs),new_pairs

In [ ]:
import pandas as pd
pair100 = ["BTC/USDT","ETH/USDT","BNB/USDT","XRP/USDT","USDC/USDT","SOL/USDT","ADA/USDT","DOGE/USDT","TRX/USDT","TON/USDT","LINK/USDT","MATIC/USDT","DOT/USDT","WBTC/USDT","LTC/USDT","DAI/USDT","SHIB/USDT","BCH/USDT","AVAX/USDT","LEO/USDT","XLM/USDT","TUSD/USDT","ATOM/USDT","OKB/USDT","XMR/USDT","UNI/USDT","ETC/USDT","FIL/USDT","HBAR/USDT","ICP/USDT","CRO/USDT","BUSD/USDT","APT/USDT","LDO/USDT","VET/USDT","NEAR/USDT","INJ/USDT","AAVE/USDT","ARB/USDT","OP/USDT","GRT/USDT","MNT/USDT","MKR/USDT","QNT/USDT","RUNE/USDT","EGLD/USDT","IMX/USDT","ALGO/USDT","STX/USDT","BSV/USDT","NEO/USDT","RNDR/USDT","AXS/USDT","THETA/USDT","SAND/USDT","MANA/USDT","XTZ/USDT","XDC/USDT","EOS/USDT","FTM/USDT","USDD/USDT","SNX/USDT","FDUSD/USDT","KAVA/USDT","TWT/USDT","BGB/USDT","FLOW/USDT","MINA/USDT","CHZ/USDT","XEC/USDT","CFX/USDT","CAKE/USDT","KCS/USDT","APE/USDT","GALA/USDT","IOTA/USDT","CRV/USDT","SUI/USDT","RPL/USDT","AR/USDT","FXS/USDT","ZEC/USDT","PEPE/USDT","XAUt/USDT","PAXG/USDT","KLAY/USDT","USDP/USDT","ROSE/USDT","ETHDYDX/USDT","BTT/USDT","CSPR/USDT","GMX/USDT","WOO/USDT","COMP/USDT","BLUR/USDT","GT/USDT","LUNC/USDT","HT/USDT","NEXO/USDT","RBTC/USDT"]
# results_file = f"../wlogs/backtests/50pairs-trendHedge.csv"
# results = pd.read_csv(results_file)
old_pairs = results['pair'].unique()
old_pairs = [p.replace("USDT","/USDT:") for p in old_pairs]
new_pairs = [p for p in pair100 if p not in old_pairs]



In [ ]:
%load_ext autoreload
%autoreload 2
import os
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from hedgeMode import HedgeMode as Strategy
def mean_k(d):
    s = 0
    c = 0
    for k,v in d.items():
        s += k*v
        c += v
    if c==0:
        return None
    return s/c

data_path='data'
files = [f for f in os.listdir(data_path) if f.endswith("5Min.csv")]
# files = ['Binance_AAVEUSDT_5Min.csv']
sl_range = [0.016,0.018,0.02,0.022,0.025 ,0.028,0.03,0.035,0.04]
# sl_range = [0.035,0.04]
market_type = 'trend'
max_fail = 20
init_money=30000
output=f'results/{datetime.now().year}-{datetime.now().month}-{datetime.now().day}'


results = []
for f in files:
    df = pd.read_csv(os.path.join(data_path,f))
    pair = f.split("_")[1]
    print(f"Pair: {pair}")
    strategy = Strategy(df,None)
    for sl in sl_range:
        vars = {
            'k':1,
            'safety_factor':1.2,
            'kpos':0,
            'start_money':1,
            'increase_factor':1,
            'normal_fail':4,
            'max_fail':max_fail,
            'tp':1+sl,
            'sl':sl,
            'leverage':100,
            'market_type':market_type,
        }
        strategy.vars = vars
        result_df = strategy.back_test()
        result_df['balance'] = result_df['long_PNL'].cumsum()+result_df['short_PNL'].cumsum()+init_money
        result_df.to_csv(f"{output}-{pair[:3]}-{strategy.__class__.__name__}-{market_type}-(sl={sl:.3f}).csv",index=False)
        print(f"Number of trades: {len(result_df)}")
        result_df['duration'] = result_df['exit_time']-result_df['entry_time']
        trades1y = result_df[result_df['entry_time']>pd.Timestamp(year=2021,month=11,day=1)]
        trades2y = result_df[result_df['entry_time']>pd.Timestamp(year=2020,month=11,day=1)]
        results.append({
            'pair':pair,
            'vars':vars,
            'start':{start},
            'end': end,
            'strategy':f"{strategy.__class__.__name__}-{vars['market_type']}",
            'profit':result_df['short_PNL'].sum()+result_df['long_PNL'].sum(),
            'init_balance':init_money,
            'final_balance':result_df.iloc[len(result_df)-1]['balance'],
            'min_balance': result_df['balance'].min(),
            'max_balance': result_df['balance'].max(),
            'average_duration':result_df['duration'].mean(),
            'n_trades':len(result_df),
            'n_fails':strategy.n_fails,
            'max_k': max(result_df['short_k'].max(),result_df['long_k'].max()),
            'max_k_1y': max(trades1y['short_k'].max(),trades1y['long_k'].max()),
            'max_k_2y': max(trades2y['short_k'].max(),trades2y['long_k'].max()),           
            'short_k_distribution': dict(result_df['short_k'].value_counts()),
            'long_k_distribution': dict(result_df['long_k'].value_counts()),
        })
        print(f"Max k (TP = {sl})= {max(result_df['short_k'].max(),result_df['long_k'].max())}")
    print()
results_file = f"results/100pairs-supertrend.csv"
results = pd.DataFrame(results)
results['tp'] = results['vars'].apply(lambda v: v['sl'])

if not  os.path.exists(results_file):
    results.to_csv(results_file,index=False)
else:
    pre_results = pd.read_csv(results_file)
    results = pd.concat([pre_results,results])
    results.to_csv(results_file,index=False)
results


In [3]:
import pandas as pd
results = pd.read_csv("results/100pairs-supertrend.csv")
results[results['pair']=='1INCHUSDT'].sort_values('pair')

,pair,vars,start,end,strategy,profit,init_balance,final_balance,min_balance,max_balance,average_duration,n_trades,n_fails,max_k,max_k_1y,max_k_2y,short_k_distribution,long_k_distribution,tp
4,1INCHUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2020-12-25 05:00:00')},2023-10-31 07:40:00,SuperTrend-trend,7565.902104,30000,37565.902104,29828.507392,37567.302104,0 days 05:59:18.461538461,3120,0,9,8.0,9.0,"{0: 1579, 1: 756, 2: 400, 3: 195, 4: 93, 5: 56...","{0: 1541, 1: 792, 2: 380, 3: 200, 4: 111, 5: 4...",0.016
5,1INCHUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2020-12-25 05:00:00')},2023-10-31 07:40:00,SuperTrend-trend,10013.838167,30000,40013.838167,27680.411441,40015.238167,0 days 06:49:14.368588609,2827,0,11,11.0,11.0,"{0: 1434, 1: 676, 2: 368, 3: 175, 4: 85, 5: 46...","{0: 1393, 1: 704, 2: 354, 3: 191, 4: 98, 5: 44...",0.018
6,1INCHUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2020-12-25 05:00:00')},2023-10-31 07:40:00,SuperTrend-trend,8451.395119,30000,38451.395119,26226.904670,38452.795119,0 days 07:43:33.968006242,2563,0,11,11.0,11.0,"{0: 1282, 1: 627, 2: 340, 3: 155, 4: 81, 5: 36...","{0: 1281, 1: 624, 2: 317, 3: 179, 4: 80, 5: 47...",0.020
7,1INCHUSDT,"{'k': 1, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2020-12-25 05:00:00')},2023-10-31 07:40:00,SuperTrend-trend,5361.307335,30000,35361.307335,29652.377024,35361.307335,0 days 08:44:36.326002587,2319,0,9,9.0,9.0,"{0: 1144, 1: 589, 2: 307, 3: 146, 4: 61, 5: 41...","{0: 1175, 1: 572, 2: 287, 3: 149, 4: 81, 5: 31...",0.022
8,1INCHUSDT,"{'k': 3, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2020-12-25 05:00:00')},2023-10-31 07:40:00,SuperTrend-trend,4337.571715,30000,34337.571715,29855.930432,34343.251715,0 days 10:06:35.527467185,2057,0,8,8.0,8.0,"{0: 1004, 1: 532, 2: 269, 3: 141, 4: 60, 5: 29...","{0: 1053, 1: 492, 2: 255, 3: 141, 4: 72, 5: 28...",0.025
9,1INCHUSDT,"{'k': 3, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2020-12-25 05:00:00')},2023-10-31 07:40:00,SuperTrend-trend,4148.277674,30000,34148.277674,29849.522432,34153.957674,0 days 11:35:30.131004366,1832,0,9,8.0,9.0,"{0: 907, 1: 460, 2: 237, 3: 115, 4: 56, 5: 35,...","{0: 925, 1: 437, 2: 228, 3: 122, 4: 69, 5: 30,...",0.028
10,1INCHUSDT,"{'k': 3, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2020-12-25 05:00:00')},2023-10-31 07:40:00,SuperTrend-trend,4131.741288,30000,34131.741288,29684.877606,34137.421288,0 days 12:41:15,1696,0,9,9.0,9.0,"{0: 845, 1: 417, 2: 214, 3: 109, 4: 53, 5: 33,...","{0: 851, 1: 406, 2: 211, 3: 117, 4: 66, 5: 23,...",0.030
11,1INCHUSDT,"{'k': 5, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2020-12-25 05:00:00')},2023-10-31 07:40:00,SuperTrend-trend,3706.767440,30000,33706.767440,29593.576141,33747.618640,0 days 15:29:44.831460674,1424,0,9,9.0,9.0,"{0: 703, 1: 345, 2: 181, 3: 92, 4: 48, 5: 29, ...","{0: 721, 1: 338, 2: 178, 3: 92, 4: 47, 5: 28, ...",0.035
12,1INCHUSDT,"{'k': 1, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2020-12-25 05:00:00')},2023-10-31 07:40:00,SuperTrend-trend,2657.762926,30000,32657.762926,29659.631206,32657.762926,0 days 18:41:01.742738589,1205,0,8,6.0,8.0,"{0: 594, 1: 303, 2: 154, 3: 75, 4: 39, 5: 23, ...","{0: 611, 1: 292, 2: 154, 3: 84, 4: 37, 5: 19, ...",0.040


In [94]:
for i,r in results.iterrows():
    filename = f'../wlogs/backtests/2023-11-5-{r["pair"][:3]}-HedgeMode-trend-(sl={r["tp"]:.3f}).csv'
    trades = pd.read_csv(filename,parse_dates=['entry_time'])
    results.at[i,'max_k'] = max(trades['short_k'].max(),trades['long_k'].max())
#     dict(trades['short_k'].value_counts())
#     print
#     results.at[i,'short_k_distribution'] = dict(trades['short_k'].value_counts())
#     results.at[i,'long_k_distribution'] = dict(trades['long_k'].value_counts())
    trades = trades[trades['entry_time']>pd.Timestamp(year=2021,month=11,day=1)]
    results.at[i,'max_k_1y'] = max(trades['short_k'].max(),trades['long_k'].max())
#     results.at[i,'short_k_distribution_1y'] = dict(trades['short_k'].value_counts())    
#     results.at[i,'long_k_distribution_1y'] = dict(trades['long_k'].value_counts())

# results['short_k_mean'] = results['short_k_distribution'].apply(mean_k)
# results['long_k_mean'] = results['long_k_distribution'].apply(mean_k)
# results['short_max_k'] = results['short_k_distribution'].apply(lambda d: max(d.keys()))
# results['long_max_k'] = results['long_k_distribution'].apply(lambda d: max(d.keys()))
# results['k_mean'] = (results['long_k_mean']+results['short_k_mean'])/2

results.sort_values(by='max_k_1y')[['pair','tp','max_k','max_k_1y']]

,pair,vars,start,end,strategy,profit,init_balance,final_balance,min_balance,max_balance,average_duration,n_trades,n_fails,short_k_mean,long_k_mean,short_max_k,long_max_k,tp,max_k,max_k_1y
178,BNBUSDT,"{'k': 3, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2017-11-06 03:50:00')},2023-10-30 08:00:00,HedgeMode-trend,4.939032e+04,30000,7.939032e+04,8.692384e+03,7.939600e+04,0 days 03:43:03.157894736,13775,0,2.051568,2.998840,13,12,0.022,13.0,7.0
90,BTCUSDT,"{'k': 4, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2017-08-17 04:00:00')},2023-10-30 07:45:00,HedgeMode-trend,3.482429e+04,30000,6.482429e+04,-9.337836e+04,6.484058e+04,0 days 12:11:35.611852522,4421,0,2.106758,2.992832,14,14,0.030,15.0,8.0
287,BNBUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{2017-11-06 03:50:00},2023-10-30 08:00:00,HedgeMode-trend,1.296386e+04,30000,4.296386e+04,2.899858e+04,4.296526e+04,0 days 08:41:04.900218011,5963,0,2.071378,2.852792,8,7,0.035,9.0,8.0
155,TRXUSDT,"{'k': 1, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2018-06-11 11:30:00')},2023-10-30 08:25:00,HedgeMode-trend,3.643592e+04,30000,6.643592e+04,2.981066e+04,6.643592e+04,0 days 03:18:42.991071428,13888,0,1.998248,2.785714,12,11,0.018,12.0,8.0
160,TRXUSDT,"{'k': 1, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2018-06-11 11:30:00')},2023-10-30 08:25:00,HedgeMode-trend,1.372102e+04,30000,4.372102e+04,1.910780e+04,4.372102e+04,0 days 08:28:25.549510337,5514,0,2.006002,3.019553,12,10,0.030,12.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,CHZUSDT,"{'k': 3, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2019-09-06 04:00:00')},2023-10-31 07:20:00,HedgeMode-trend,2.015730e+05,30000,2.315730e+05,-5.239392e+05,2.315787e+05,0 days 01:34:50.310627201,21859,0,2.071142,3.046529,14,13,0.018,17.0,17.0
35,AUDIOUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2020-10-23 19:00:00')},2023-10-31 08:00:00,HedgeMode-trend,3.665408e+05,30000,3.965408e+05,-1.207968e+06,3.965422e+05,0 days 01:02:32.501701837,23504,0,1.965351,3.014403,16,14,0.016,18.0,18.0
162,1INCHUSDT,"{'k': 3, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2020-12-25 05:00:00')},2023-10-31 07:40:00,HedgeMode-trend,7.094404e+05,30000,7.394404e+05,-2.782197e+06,7.394461e+05,0 days 01:43:17.858486470,13822,0,2.088190,3.136307,17,15,0.018,19.0,19.0
24,XLMUSDT,"{'k': 6, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2018-05-31 09:30:00')},2023-10-30 09:00:00,HedgeMode-trend,6.240044e+05,30000,6.540044e+05,-2.848348e+06,6.541005e+05,0 days 04:01:19.250670589,11557,0,1.999656,3.050360,18,16,0.022,19.0,19.0


In [95]:
results.sort_values(by='max_k_1y')[['pair','tp','max_k','max_k_1y']]

,pair,tp,max_k,max_k_1y
178,BNBUSDT,0.022,13.0,7.0
90,BTCUSDT,0.030,15.0,8.0
287,BNBUSDT,0.035,9.0,8.0
155,TRXUSDT,0.018,12.0,8.0
160,TRXUSDT,0.030,12.0,8.0
...,...,...,...,...
253,CHZUSDT,0.018,17.0,17.0
35,AUDIOUSDT,0.016,18.0,18.0
162,1INCHUSDT,0.018,19.0,19.0
24,XLMUSDT,0.022,19.0,19.0


In [ ]:
resu

In [97]:
for i,r in results.iterrows():
    filename = f'../wlogs/backtests/2023-11-5-{r["pair"][:3]}-HedgeMode-trend-(sl={r["tp"]:.3f}).csv'
    trades = pd.read_csv(filename,parse_dates=['entry_time'])
    trades = trades[trades['entry_time']>pd.Timestamp(year=2020,month=11,day=1)]
    results.at[i,'max_k_2y'] = max(trades['short_k'].max(),trades['long_k'].max())
results.sort_values(by='max_k_2y')[results['tp']==0.02][['pair','tp','max_k','max_k_1y','max_k_2y']]


/tmp/ipykernel_764323/1470700745.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  results.sort_values(by='max_k_2y')[results['tp']==0.02][['pair','tp','max_k','max_k_1y','max_k_2y']]


,pair,tp,max_k,max_k_1y,max_k_2y
86,BTCUSDT,0.02,10.0,9.0,10.0
191,XMRUSDT,0.02,11.0,10.0,10.0
156,TRXUSDT,0.02,12.0,11.0,11.0
142,ETHUSDT,0.02,11.0,11.0,11.0
121,LINKUSDT,0.02,14.0,11.0,11.0
114,KAVAUSDT,0.02,12.0,11.0,11.0
135,CHRUSDT,0.02,11.0,11.0,11.0
233,ADAUSDT,0.02,14.0,11.0,11.0
212,SXPUSDT,0.02,11.0,9.0,11.0
198,EOSUSDT,0.02,11.0,10.0,11.0


In [100]:
results.sort_values(by='max_k_2y')[['pair','tp','max_k','max_k_1y','max_k_2y','final_balance','min_balance','max_balance','n_trades']]

,pair,tp,max_k,max_k_1y,max_k_2y,final_balance,min_balance,max_balance,n_trades
288,BNBUSDT,0.040,8.0,8.0,8.0,3.958995e+04,2.995499e+04,3.959135e+04,4539
160,TRXUSDT,0.030,12.0,8.0,8.0,4.372102e+04,1.910780e+04,4.372102e+04,5514
159,TRXUSDT,0.028,11.0,9.0,9.0,4.470691e+04,2.560950e+04,4.470691e+04,6249
174,DASHUSDT,0.030,11.0,9.0,9.0,4.478154e+04,2.834194e+04,4.478154e+04,5448
180,BNBUSDT,0.028,10.0,9.0,9.0,5.235630e+04,2.881433e+04,5.245237e+04,8965
...,...,...,...,...,...,...,...,...,...
162,1INCHUSDT,0.018,19.0,19.0,19.0,7.394404e+05,-2.782197e+06,7.394461e+05,13822
24,XLMUSDT,0.022,19.0,19.0,19.0,6.540044e+05,-2.848348e+06,6.541005e+05,11557
77,ZILUSDT,0.016,19.0,12.0,19.0,6.983870e+05,-2.836226e+06,6.983884e+05,26278
79,ZILUSDT,0.020,20.0,10.0,20.0,1.367176e+06,-6.349946e+06,1.367176e+06,17738


In [45]:
results = results.drop(columns=['short_max_k','long_max_k'])

In [16]:
result_df = pd.read_csv('../wlogs/backtests/2023-11-5-ETC-HedgeMode-trend-(sl=0.025).csv')
print(f"max short k={result_df['short_k'].max()}")
print(f"max long k={result_df['long_k'].max()}")

max short k=10
max long k=11


In [18]:
short_k_dist = result_df['short_k'].value_counts()
# print(short_k_dist)
short_k_dist.at[len(short_k_dist)]=0
# print(short_k_dist)
short_k_dist = short_k_dist.diff(-1).dropna()[1:].astype(int)
# print(short_k_dist)
long_k_dist = result_df['long_k'].value_counts()
print(long_k_dist)
long_k_dist.at[len(long_k_dist)]=0
print(long_k_dist)
long_k_dist = short_k_dist.diff(-1).dropna()[1:].astype(int)
print(long_k_dist)


long_k
0     4850
1     2439
2     1222
3      652
4      320
5      176
6       72
7       41
8       13
9        7
11       1
Name: count, dtype: int64
long_k
0     4850
1     2439
2     1222
3      652
4      320
5      176
6       72
7       41
8       13
9        7
11       0
Name: count, dtype: int64
short_k
2    382
3     64
4    134
5     14
6     36
7      1
8     12
9      0
Name: count, dtype: int64
